# Data Import

In [81]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

df = pd.read_stata('assignment8.dta')
display(df.info(), df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820 entries, 0 to 819
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   store         820 non-null    float32
 1   chain         820 non-null    float32
 2   co_owned      820 non-null    float32
 3   state         820 non-null    float32
 4   southj        820 non-null    float32
 5   centralj      820 non-null    float32
 6   northj        820 non-null    float32
 7   pa1           820 non-null    float32
 8   pa2           820 non-null    float32
 9   shore         820 non-null    float32
 10  ncalls        571 non-null    float32
 11  empft         802 non-null    float32
 12  emppt         806 non-null    float32
 13  nmgrs         808 non-null    float32
 14  wage_st       779 non-null    float32
 15  inctime       723 non-null    float32
 16  firstinc      697 non-null    float32
 17  meals         809 non-null    float32
 18  open          809 non-null    

None

,store,chain,co_owned,state,southj,centralj,northj,pa1,pa2,shore,...,firstinc,meals,open,hoursopen,pricesoda,pricefry,priceentree,nregisters,nregisters11,time
0,46.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,NaN,2.0,6.5,16.5,1.03,1.03,0.52,3.0,3.0,0.0
1,49.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,NaN,2.0,10.0,13.0,1.01,0.90,2.35,4.0,3.0,0.0
2,506.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.37,2.0,11.0,10.0,0.95,0.74,2.33,3.0,3.0,0.0
3,56.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.10,2.0,10.0,12.0,0.87,0.82,1.79,2.0,2.0,0.0
4,61.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.15,3.0,10.0,12.0,0.87,0.77,1.65,2.0,2.0,0.0
5,62.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.07,2.0,10.0,12.0,0.87,0.77,0.95,2.0,2.0,0.0
6,445.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.10,2.0,6.0,18.0,1.04,0.88,0.94,3.0,3.0,0.0
7,451.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.25,2.0,0.0,24.0,1.05,0.84,0.96,6.0,4.0,0.0
8,455.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.25,1.0,11.0,10.0,0.73,0.73,2.32,2.0,2.0,0.0
9,458.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.15,1.0,11.0,10.0,0.94,0.73,2.32,4.0,4.0,0.0


# Data Cleaning

In [111]:
## DATA PREPARATION

chain_dict = {
    1.0: "B",
    2.0: "K",
    3.0: "R",
    4.0: "W"
}

# Data Preparation to Match Sample Size n = 357 from Table 4 Notes of Card & Kreuger (1994)
# - this uses FTE = full time + 0.5 * part time + managers
# - store filter: existence FTE = full time + 0.5 * part time AND nmgrs is not missing in second period
# - this method allows for missingness in managers data only for the second period to match the desired n = 357
df_reduced_match_n = (
    df.filter(items = ['store', 'time', 'empft', 'emppt', 'nmgrs', 'wage_st', 'state', 'chain', 'co_owned'])
      .replace({"chain": chain_dict})

      ## matching to n
      .assign(fte = lambda x: x['empft'] + 0.5*x['emppt'])
      .groupby("store")
      .filter(lambda x: (not x.loc[x['time'] == 1, 'nmgrs'].isna().any()) and 
                        (((x["fte"].notna()) & (x["wage_st"].notna())).all()))
      .assign(fte = lambda x: x['fte'] + x['nmgrs'].fillna(0))
      ## -------------

      .pipe(lambda x: x.join(pd.get_dummies(x['chain'], prefix = 'chain', drop_first = True, dtype = 'float')))
      .assign(d_E = lambda x: x.groupby('store')['fte'].transform(lambda y: y.diff()))
      .dropna(subset = 'd_E')
      .drop(columns = ['fte', 'empft', 'emppt', 'nmgrs', 'chain', 'time', 'wage_st'])
)

display(df_reduced_match_n.shape, df_reduced_match_n.d_E.describe().iloc[0:3,], df_reduced_match_n.head(5))

# Data Preparation to Match FTE Definition for Tables 3 and 4 of Card & Kreuger (1994)
# - this uses FTE = full time + 0.5 * part time + managers
# - store filter: existence FTE = full time + 0.5 * part time + nmgrs 
# - this methods follows the FTE definition in detail and retains a smaller set of the sample (n = 351)

df_reduced_match_fte = (
    df.filter(items = ['store', 'time', 'empft', 'emppt', 'nmgrs', 'wage_st', 'state', 'chain', 'co_owned'])
      .replace({"chain": chain_dict})
      
      ## matching to fte
      .assign(fte = lambda x: x['empft'] + 0.5*x['emppt'] + x['nmgrs'])
      .groupby("store")
      .filter(lambda x: ((x["fte"].notna()) & (x["wage_st"].notna())).all())
      ## ---------------

      .pipe(lambda x: x.join(pd.get_dummies(x['chain'], prefix = 'chain', drop_first = True, dtype = 'float')))
      .assign(d_E = lambda x: x.groupby('store')['fte'].transform(lambda y: y.diff()))
      .dropna(subset = 'd_E')
      .drop(columns = ['fte', 'empft', 'emppt', 'nmgrs', 'chain', 'time', 'wage_st'])
)

display(df_reduced_match_fte.shape, df_reduced_match_fte.d_E.describe().iloc[0:3,], df_reduced_match_fte.head(5))

(357, 7)

count    357.000000
mean       0.027871
std        8.770654
Name: d_E, dtype: float64

,store,state,co_owned,chain_K,chain_R,chain_W,d_E
413,56.0,0.0,1.0,0.0,0.0,1.0,-14.0
414,61.0,0.0,1.0,0.0,0.0,1.0,11.5
416,445.0,0.0,0.0,0.0,0.0,0.0,-41.5
417,451.0,0.0,0.0,0.0,0.0,0.0,13.0
418,455.0,0.0,1.0,1.0,0.0,0.0,0.0


(351, 7)

count    351.000000
mean      -0.030057
std        8.747621
Name: d_E, dtype: float64

,store,state,co_owned,chain_K,chain_R,chain_W,d_E
413,56.0,0.0,1.0,0.0,0.0,1.0,-14.0
414,61.0,0.0,1.0,0.0,0.0,1.0,11.5
416,445.0,0.0,0.0,0.0,0.0,0.0,-41.5
417,451.0,0.0,0.0,0.0,0.0,0.0,13.0
418,455.0,0.0,1.0,1.0,0.0,0.0,0.0


## Note: Validation of n = 357

I show that when we allow missingness for managers, the difference between the dataset of those with managers missingsness and managers non-missingness reflects a difference that results in the matching of the n = 357 sample size in Card and Krueger (1994). Specifically, the resulting stores with missingness in managers data shows that only one store showed missingness in the second period but not in the first. We suspect this is an unstated rule that Card and Krueger may have implemented when filtering. The rule: keep stores with missingness only in the first period.

In [94]:
yes_mgrs = df.filter(items = ['store', 'time', 'empft', 'emppt', 'nmgrs', 'wage_st'])\
  .assign(fte = lambda x: x['empft'] + 0.5*x['emppt'] + x['nmgrs'])\
  .drop(columns = ['empft', 'emppt', 'nmgrs'])\
  .groupby("store").filter(lambda x: ((x["fte"].notna()) & (x["wage_st"].notna())).all())\
  .assign(d_E = lambda x: x.groupby('store')['fte'].transform(lambda y: y.diff()))\
  .dropna(subset = 'd_E')\
  .drop(columns = ['time'])

no_mgrs = df.filter(items = ['store', 'time', 'empft', 'emppt', 'wage_st'])\
  .assign(fte = lambda x: x['empft'] + 0.5*x['emppt'])\
  .drop(columns = ['empft', 'emppt'])\
  .groupby("store").filter(lambda x: ((x["fte"].notna()) & (x["wage_st"].notna())).all())\
  .assign(d_E = lambda x: x.groupby('store')['fte'].transform(lambda y: y.diff()))\
  .dropna(subset = 'd_E')\
  .drop(columns = ['time'])

display(no_mgrs.store.nunique() - yes_mgrs.store.nunique(),
        set(no_mgrs.store.unique()) - set(yes_mgrs.store.unique()),
        set(yes_mgrs.store.unique()) - set(no_mgrs.store.unique()))

7

{np.float32(47.0),
 np.float32(63.0),
 np.float32(80.0),
 np.float32(87.0),
 np.float32(313.0),
 np.float32(362.0),
 np.float32(403.0)}

set()

In [99]:
# See store 313.0
df[df['store'].isin([47, 63, 80, 87, 313, 362, 403])].sort_values('store')[['store','empft', 'emppt', 'nmgrs']]

,store,empft,emppt,nmgrs
33,47.0,25.0,15.0,NaN
443,47.0,0.0,45.0,2.0
195,63.0,26.0,6.0,NaN
605,63.0,15.0,10.0,4.0
208,80.0,4.5,20.0,NaN
618,80.0,11.0,14.0,3.0
211,87.0,0.0,15.0,NaN
621,87.0,0.0,30.0,5.0
344,313.0,11.0,9.0,4.0
754,313.0,0.0,30.0,NaN


# Modeling

## Using Matched n

In [112]:
model_i_n = smf.ols('d_E ~ state', 
                      data = df_reduced_match_n).fit()

print(model_i_n.summary())

                            OLS Regression Results                            
Dep. Variable:                    d_E   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     4.320
Date:                Tue, 02 Dec 2025   Prob (F-statistic):             0.0384
Time:                        20:43:49   Log-Likelihood:                -1279.1
No. Observations:                 357   AIC:                             2562.
Df Residuals:                     355   BIC:                             2570.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.9701      1.067     -1.847      0.0

In [113]:
model_ii_n = smf.ols('d_E ~ state + co_owned + chain_K + chain_R + chain_W', 
                      data = df_reduced_match_n).fit()

print(model_ii_n.summary())

                            OLS Regression Results                            
Dep. Variable:                    d_E   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     1.640
Date:                Tue, 02 Dec 2025   Prob (F-statistic):              0.149
Time:                        20:44:03   Log-Likelihood:                -1277.1
No. Observations:                 357   AIC:                             2566.
Df Residuals:                     351   BIC:                             2590.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.5564      1.199     -1.298      0.1

## Using Matched FTE

In [114]:
model_i_fte = smf.ols('d_E ~ state', 
                      data = df_reduced_match_fte).fit()

print(model_i_fte.summary())

                            OLS Regression Results                            
Dep. Variable:                    d_E   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     3.658
Date:                Tue, 02 Dec 2025   Prob (F-statistic):             0.0566
Time:                        20:45:14   Log-Likelihood:                -1257.0
No. Observations:                 351   AIC:                             2518.
Df Residuals:                     349   BIC:                             2526.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.8788      1.073     -1.751      0.0

In [116]:
model_ii_fte = smf.ols('d_E ~ state + co_owned + chain_K + chain_R + chain_W', 
                      data = df_reduced_match_fte).fit()

print(model_ii_fte.summary())

                            OLS Regression Results                            
Dep. Variable:                    d_E   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.436
Date:                Tue, 02 Dec 2025   Prob (F-statistic):              0.211
Time:                        20:45:32   Log-Likelihood:                -1255.2
No. Observations:                 351   AIC:                             2522.
Df Residuals:                     345   BIC:                             2546.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.4500      1.210     -1.198      0.2